In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ebmdatalab import bq, maps, charts

In [2]:
sql= '''
SELECT
  pct,
  register_size AS dementia_register,
  SUM(items) AS items,
  SUM(actual_cost) AS cost
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
INNER JOIN
  ebmdatalab.hscic.qof_ccgs_1415 AS ccg ##1415 register - update if we do anything with it
ON
  presc.pct = ccg.ccg_code
WHERE
  (bnf_code LIKE "0409010M%" OR ##Memantine
    bnf_code LIKE "0411000D0%" OR ##Donepezil
    bnf_code LIKE "0411000F0%" OR  ##Galantamine
    bnf_code LIKE "0411000E0%")
  AND (month BETWEEN TIMESTAMP('2018-01-01')
    AND TIMESTAMP('2018-12-01'))
    AND ccg.indicator_code = "DEM"
   GROUP BY
   pct
 '''

df_dementia = bq.cached_read(sql, csv_path='dementia.csv')
df_dementia.head(10)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


GenericGBQException: Reason: 400 SELECT list expression references column register_size which is neither grouped nor aggregated at [4:3]

In [ ]:
df_dementia['item per dem patient'] = df_dementia['items'] / df_dementia['dementia_register']
df_dementia.head(5)

In [ ]:
plt.figure(figsize=(12, 7))
maps.ccg_map(df_dementia, title="Dementia Medicines per Patient on Dementia Register", column='item per dem patient', 
             separate_london=True)
plt.savefig('map.png', format='png', dpi=300,bbox_inches='tight')
plt.show()